# An Example of Cython and Numpy

> see https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial

## Write the code

In [3]:
# cython: infer_types=True
from __future__ import print_function
import numpy as np
cimport cython
#The primitive types supported are tied closely to those in C:
ctypedef float MY_DTYPE # means float in C type

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments and the return value.
cdef float clip(float a, float min_value, float max_value):
    return min(max(a, min_value), max_value)

@cython.boundscheck(False)
@cython.wraparound(False)

#For extra speed gains, if you know that the NumPy arrays 
#you are providing are contiguous in memory, you can 
#declare the memoryview as contiguous. 
    
#We give an example on an array that has 3 dimensions. 
#If you want to give Cython the information that the 
#data is C-contiguous you have to declare the memoryview like this:
#cdef int [:,:,::1] a


#The cdef statement is used to declare C variables, either local or module-level:
def writeKT15FalseColor(MY_DTYPE [:,::1] array_disp):
    clr_map = np.array(
            [[0,0,0,114],[0,0,1,185],[1,0,0,114],[1,0,1,174],
             [0,1,0,114],[0,1,1,185],[1,1,0,114],[1,1,1,0],], 
            dtype=np.float32)
    
    cdef MY_DTYPE[:,::1] clr_map_view = clr_map
    
    cdef float mySum = .0
    cdef Py_ssize_t i, j, k
    
    for i in range(0,8):
        mySum += clr_map_view[i][3]
    
    weights = np.zeros((8), dtype = np.float32)
    cumsum  = np.zeros((8), dtype = np.float32)
    cdef MY_DTYPE [::1] weights_view = weights
    cdef MY_DTYPE [::1] cumsum_view = cumsum
    
    for i in range(0,7):
        weights_view[i] = mySum / clr_map_view[i,3]
        cumsum_view[i+1] = cumsum_view[i] + clr_map_view[i,3] / mySum
    #print ('weights: ', weights)
    #print ('cumsum: ', cumsum)

    cdef Py_ssize_t h = array_disp.shape[0]
    cdef Py_ssize_t w = array_disp.shape[1]
    cdef float max_val = -1.0
    for i in range(h):
        for j in range(w):
            if max_val < array_disp[i,j]:
                max_val = array_disp[i,j]
    #print ("max_val = ", max_val)

    disp_out = np.zeros([h, w, 3], dtype=np.float32)
    cdef MY_DTYPE[:,:,::1] disp_out_view = disp_out
    cdef MY_DTYPE tmp_w
    for i in range(0, h):
        for j in range(0, w):
            # get normalized value
            val = clip(array_disp[i,j]/ max_val, .0, 1.0)
            # find bin;
            for k in range(0,7):
                if val < cumsum_view[k+1]:
                    #print (i,j,k, val, cumsum[k+1])
                    break
            # compute red/green/blue values
            tmp_w = 1.0 - (val-cumsum_view[k]) * weights_view[k]
            # r, g, b
            disp_out_view[i,j, 0] = (tmp_w*clr_map_view[k, 0] + (1.0-tmp_w)*clr_map_view[k+1,0])*255.0
            disp_out_view[i,j, 1] = (tmp_w*clr_map_view[k, 1] + (1.0-tmp_w)*clr_map_view[k+1,1])*255.0
            disp_out_view[i,j, 2] = (tmp_w*clr_map_view[k, 2] + (1.0-tmp_w)*clr_map_view[k+1,2])*255.0
            #if i == 200 and j == 300:
                #print ('disp_out[200,300] = ', disp_out[i,j,:])
                #print (i,j,k, val, cumsum[k+1])
    return disp_out

SyntaxError: invalid syntax (<ipython-input-3-f737de3de595>, line 4)

In [2]:
# !/usr/bin/env python3
# -*-coding:utf-8-*-
# @file: writeKT15ErrorLogColor.pyx
# @brief:
# @author: Changjiang Cai, ccai1@stevens.edu, caicj5351@gmail.com
# @version: 0.0.1
# @creation date: 28-10-2019
# @last modified: Mon 28 Oct 2019 07:21:09 PM EDT

# cython: infer_types=True
from __future__ import print_function
import numpy as np
cimport cython
from libc.math cimport fabs # fabs() from <cmath>;
#The primitive types supported are tied closely to those in C:
ctypedef float MY_DTYPE # means float in C type

@cython.boundscheck(False)
@cython.wraparound(False)

#For extra speed gains, if you know that the NumPy arrays 
#you are providing are contiguous in memory, you can 
#declare the memoryview as contiguous. 
    
#We give an example on an array that has 3 dimensions. 
#If you want to give Cython the information that the 
#data is C-contiguous you have to declare the memoryview like this:
#cdef int [:,:,::1] a

#The cdef statement is used to declare C variables, either local or module-level:
#NOTE:The code is adopted from the C/C++ code provided by KITTI15 official website;
def writeKT15ErrorDispLogColor(
        MY_DTYPE [:,::1] array_disp, # prediciton
        MY_DTYPE [:,::1] array_disp_gt # ground truth
        ):
    # size: [10,5]
    log_clr_map = np.array(
           [[0,0.0625,49,54,149],
            [0.0625,0.125,69,117,180],
            [0.125,0.25,116,173,209],
            [0.25,0.5,171,217,233],
            [0.5,1,224,243,248],
            [1,2,254,224,144],
            [2,4,253,174,97],
            [4,8,244,109,67],
            [8,16,215,48,39],
            [16,1000000000.0,165,0,38]
           ],dtype=np.float32)
    
    cdef MY_DTYPE[:,::1] log_clr_map_view = log_clr_map
    cdef Py_ssize_t h = array_disp.shape[0]
    cdef Py_ssize_t w = array_disp.shape[1]
    cdef Py_ssize_t i, v,u,v2,u2
    cdef float val_red = .0, val_gre = .0, val_blu = .0, d_err, d_mag, n_err
    disp_err = np.zeros([h, w, 3], dtype=np.float32)
    cdef MY_DTYPE[:,:,::1] disp_err_view = disp_err
    for v in range(1, h-1):
        for u in range(1, w-1):
            if array_disp_gt[v,u] > 0: # if valid
                d_err = fabs(array_disp[v,u] - array_disp_gt[v,u])
                d_mag = fabs(array_disp_gt[v,u])
                n_err = min(d_err / 3.0, 20.0*d_err/d_mag)
                for i in range(0, 10):
                    if (n_err >= log_clr_map_view[i,0]) and (n_err < log_clr_map_view[i,1]):
                        val_red = log_clr_map_view[i,2]
                        val_gre = log_clr_map_view[i,3]
                        val_blu = log_clr_map_view[i,4]
                disp_err_view[v,u,0] = val_red
                disp_err_view[v,u,1] = val_gre
                disp_err_view[v,u,2] = val_blu
    return disp_err

SyntaxError: invalid syntax (<ipython-input-2-a2eb2b88b9c0>, line 13)

## Write a setup file

In [4]:
from distutils.core import setup
from Cython.Build import cythonize
from distutils.extension import Extension

ext_modules = [Extension('writeKT15FalseColor', ['writeKT15FalseColor.pyx'])]

setup(
        ext_modules = cythonize(ext_modules)
    )

ValueError: 'writeKT15FalseColor.pyx' doesn't match any files

## Compile it

```
MY_PYTHON=python3
rm ./__init__.py
rm ./__init__.pyc
rm *.so
rm *.c
$MY_PYTHON ./setup_KT15ErrLogColor.py clean
rm -rf build
$MY_PYTHON ./setup_KT15ErrLogColor.py build_ext --inplace


$MY_PYTHON ./setup_KT15FalseColor.py clean
rm -rf build
$MY_PYTHON ./setup_KT15FalseColor.py build_ext --inplace

cp ../__init__.py ./
```

## Use the `.so` lib in Python Code

In [5]:
from cython import writeKT15FalseColor as KT15FalseClr
from cython import writeKT15ErrorLogColor as KT15LogClr
if __name__ == "__main__":
        since = time.time()
        for i in range(5,6):
            disp_gt = readPFM('/data/ccjData/datasets/KITTI-2015/training/disp_occ_0_pfm/%06d_10.pfm' %i)
            disp_gt[disp_gt == np.inf] = .0
            #disp = readPFM('/media/ccjData2/research-projects/GCNet/results/gcnet-F8-RMSp-sfF3k-epo31-4dsConv-k5-testKT15/disp-epo-030/%06d_10.pfm' %i)
            disp = readPFM('/home/ccj/atten-stereo/results/ganet-sfepo10-kt15epo100/val-30/%06d_10.pfm' %i)
            disp[disp == np.inf] = .0
            rst_disp = KT15FalseClr.writeKT15FalseColor(disp).astype(np.uint8)
            rst_disp_gt = KT15FalseClr.writeKT15FalseColor(disp_gt).astype(np.uint8)
            show_uint8(rst_disp, title = 'disp_kt15_false_color')
            show_uint8(rst_disp_gt, title = 'disp_gt_kt15_false_color')
            err = np.abs(disp_gt - disp)
            rst_err = KT15LogClr.writeKT15ErrorDispLogColor(disp, disp_gt)
            show_uint8(rst_err, title = 'err_kt15_log_color')
            show(err, title = 'err gray color')

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}s'.format(time_elapsed))

ImportError: cannot import name 'writeKT15FalseColor' from 'cython' (/usr/local/lib/python3.7/site-packages/cython.py)

## Note:

The above cannot be run, but the code itself is right.